# Backtesting with the Backtesting Library

This notebook demonstrates how to run a backtest using the Python `backtesting` library. We define a strategy based on our model's predictions.

In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy

/opt/anaconda3/envs/gold-env/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Define the Strategy

In this strategy, if the prediction is positive, we enter a long trade; if negative, a short trade.
We set a take profit and stop loss based on 70% of the predicted move and a risk-reward ratio of 1:2.

In [ ]:
# Define a strategy that adds a new trade each day based on the prediction,
class Strategy(Strategy):
    trade_pct = 0.7       # Use 70% of the predicted move for TP calculation.
    risk_reward = 2.0     # Stop loss is set as (trade_pct * pred) / risk_reward.
    trade_size = 1        # Explicitly set a trade size of 1 unit

    def init(self):
        # No indicators are needed for this strategy
        pass

    def next(self):
        # Get today's prediction and today's open price.
        pred = self.data.prediction[-1]
        open_price = self.data.Open[-1]
        
        # Only trade if prediction is non-zero
        if pred - open_price > 0:
            # For a long trade:
            target_price = open_price + self.trade_pct * pred
            stop_price = open_price - (self.trade_pct * pred) / self.risk_reward
            self.buy(size=self.trade_size, sl=stop_price, tp=target_price)
        elif pred - open_price < 0:
            # For a short trade:
            target_price = open_price - self.trade_pct * abs(pred)
            stop_price = open_price + (self.trade_pct * abs(pred)) / self.risk_reward
            self.sell(size=self.trade_size, sl=stop_price, tp=target_price)


class Strategy(Strategy):
    trade_pct = 0.7       # Use 70% of the predicted move for TP calculation.
    risk_reward = 2.0     # Stop loss is set as (trade_pct * predicted_move) / risk_reward.
    trade_size = 1        # Trade size

    def init(self):
        self.tradeTaken = False
        self.last_trade_date = None

    def next(self):
        current_datetime = self.data.index[-1]
        current_date = current_datetime.date()
        
        if self.last_trade_date != current_date:
            self.tradeTaken = False
            self.last_trade_date = current_date

        if not self.tradeTaken:
            open_price = self.data.Open[-1]
            pred = self.data.prediction[-1]
            
            predicted_move = pred - open_price

            if predicted_move > 0:
                # Long trade
                target_price = open_price + self.trade_pct * predicted_move
                stop_price = open_price - (self.trade_pct * predicted_move) / self.risk_reward
                print(f"{current_date} LONG -> Open: {open_price:.2f}, Pred: {pred:.2f}, Move: {predicted_move:.2f}, "
                      f"TP: {target_price:.2f}, SL: {stop_price:.2f}")
                self.buy(size=self.trade_size, sl=stop_price, tp=target_price)
                self.tradeTaken = True

            elif predicted_move < 0:
                # Short trade
                target_price = open_price - self.trade_pct * abs(predicted_move)
                stop_price = open_price + (self.trade_pct * abs(predicted_move)) / self.risk_reward
                print(f"{current_date} SHORT -> Open: {open_price:.2f}, Pred: {pred:.2f}, Move: {predicted_move:.2f}, "
                      f"TP: {target_price:.2f}, SL: {stop_price:.2f}")
                self.sell(size=self.trade_size, sl=stop_price, tp=target_price)
                self.tradeTaken = True
            else:
                print(f"{current_date}: No trade signal; predicted move is zero.")

## Load and Prepare the Data

We load our backtest data from the CSV file and rename the columns accordingly (as required by the library).

In [7]:
# Load backtest data from CSV
df = pd.read_csv('../data/processed/backtest_data_final.csv', parse_dates=['date'])
df.set_index('date', inplace=True)
df.index.name = 'Date'

# Rename columns to match the backtesting library's requirements
df.rename(columns={
    'date': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)


# Verify the first few rows
df.head()

,Open,High,Low,Close,Volume,target,prediction
Date,,,,,,,
2023-04-05,2022.099976,2033.800049,2013.599976,2020.900024,525,2033.583070,2029.578891
2023-04-06,2022.199951,2023.300049,2005.000000,2011.900024,0,2005.000850,2031.754624
2023-04-10,2000.000000,2006.599976,1984.000000,1989.099976,652,1983.822400,2009.275359
2023-04-11,1991.400024,2006.500000,1991.400024,2004.800049,68,2006.565210,1999.763136
2023-04-12,2005.199951,2025.699951,2005.199951,2010.900024,172,2025.558869,2013.640832


## Run the Backtest

We now create a Backtest object using our data and strategy. We set the initial cash and commission. Then, we run the backtest and plot the results.

In [ ]:
# Create the Backtest object
bt = Backtest(df, Strategy, cash=100000, commission=0.001)

# Run the backtest and print statistics
stats = bt.run()
print(stats)

# Plot the backtest results
bt.plot(filename="../data/output/bt_result.html")

Start                     2023-04-05 00:00:00
End                       2025-02-26 00:00:00
Duration                    693 days 00:00:00
Exposure Time [%]                    97.89916
Equity Final [$]                 146189.38082
Equity Peak [$]                   147744.3735
Commissions [$]                      527.1595
Return [%]                           46.18938
Buy & Hold Return [%]                44.33173
Return (Ann.) [%]                    22.26675
Volatility (Ann.) [%]                15.51809
CAGR [%]                             14.80727
Sharpe Ratio                          1.43489
Sortino Ratio                         2.54721
Calmar Ratio                           2.7979
Alpha [%]                             6.83479
Beta                                  0.88773
Max. Drawdown [%]                    -7.95839
Avg. Drawdown [%]                    -1.87747
Max. Drawdown Duration      169 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                          

GridPlot(id='p1681', ...)

### Strategy results and takeaways
- We can see relevant metrics about our strategy like # of trades taken, Win rate, Market exposure, etc.
- Even though the results are positive, the strategy wasn't implemented properly by the library. 
- Different methods, calculations and entry types were tested but non of them returned the expected results
- Further research about the library's implementation is needed in order to use the strategy accordingly